In [1]:
import torch
import numpy as np
from web3 import Web3
import abi
from web3.middleware import geth_poa_middleware
import model

### Declarar web3 con binance smart chain Testnet
### Definir private key de la wallet origen (realiza gastos)


In [2]:
w3 = Web3(Web3.HTTPProvider('https://data-seed-prebsc-1-s1.binance.org:8545'))
w3.middleware_onion.inject(geth_poa_middleware, layer=0) 

private_key="db95cd3ee5421326ab4cca7edbccc1ae3a34f77d13fa355f8a23cfdfda11ba26"
account=w3.eth.account.from_key("db95cd3ee5421326ab4cca7edbccc1ae3a34f77d13fa355f8a23cfdfda11ba26")

### Deploy del contrato "Neural network", Se calcula el gas y se hace deploy de la transaccion firmada por la cuenta de origen

In [3]:
neuralNetwork_contract = w3.eth.contract(abi=abi.nnAbi, bytecode=abi.nnBin)
estimateGas=neuralNetwork_contract.constructor().estimateGas() + 100000
deploy_trans=neuralNetwork_contract.constructor().buildTransaction({'gas':estimateGas ,'nonce': w3.eth.get_transaction_count(account.address),'gasPrice': w3.toWei(10, 'gwei')})
signed_txn = w3.eth.account.sign_transaction(deploy_trans, private_key=private_key)
send=w3.eth.sendRawTransaction(signed_txn.rawTransaction) 
print(send.hex())
contract_nn_deployed = w3.eth.wait_for_transaction_receipt(send.hex()).contractAddress

0x36cdb55b79790cc9df09f1711ad165ec54be28881d46510aba575b648849bff9


### Se carga el modelo a guardar en conjunto con su state dict, además de cargar el contrato "neural network" deployeado anteriormente

In [4]:
contract=w3.eth.contract(address=contract_nn_deployed, abi=abi.nnAbi )
modelo=model.nnblock()
args_model= modelo.state_dict()

### Recorrer cada capa para guardar el weight y bias 

In [ ]:
for name in range(0,len(args_model),2):
    keys=list(args_model.keys())
    nonce= w3.eth.get_transaction_count(account.address)  
    layer_name= (keys[name][0:keys[name].find(".")])
    layer_weight =np.array(((args_model[layer_name+".weight"].reshape(-1)) * (10**10)).detach() ,dtype=int).tolist()
    layer_bias = np.array(((args_model[layer_name + ".bias"].reshape(-1)) * (10**10)).detach() ,dtype=int).tolist()
    
    a=contract.functions.NewLayer(layer_weight,layer_bias,layer_name).buildTransaction({'from': account.address,'nonce': nonce ,'gasPrice': 20000000000})
    signed_txn = w3.eth.account.sign_transaction(a, private_key=private_key)
    send=w3.eth.sendRawTransaction(signed_txn.rawTransaction)
    w3.eth.wait_for_transaction_receipt(send.hex())
    print(send.hex())

0x731c7b3a840872b4965ec07d03ae09a989c4f14a342a0d54d42aff1a0795ea6a


In [ ]:
contract_nn_deployed 

### Como prueba, se pueden obtener el nombre de las capas añadidas en el paso anterior

In [ ]:
contract.functions.getLayersNames().call()

### Visualizar el ultimo timestamp en el que se modificó la capa "conv1"

In [ ]:
from datetime import datetime
dt_object = datetime.fromtimestamp(contract.functions.getLastModify("conv1").call())
print("fecha de modificacion de la capa : ", dt_object)

### Visualizar los pesos y bias de una capa

In [ ]:
contract.functions.getLayerWeight("conv1").call()

In [ ]:
contract.functions.getLayerBias("conv1").call()

### NEXT STEP, cargar pesos desde etherum 

In [ ]:
modelo.conv1.state_dict()['weight'].shape